### 7.1.2 层次分析法的案例


**例7.1**：某日从三条河流的基站处抽检水样，得到了水质的四项检测指标如表5.1所示。请根据提供数据对三条河流的水质进行评价。其中，DO代表水中溶解氧含量，越大越好；CODMn表示水中高锰酸盐指数，NH3-N表示氨氮含量，这两项指标越小越好；pH值没有量纲，在6~9区间内较为合适。

**例7.1的数据**

| **地点名称**   | **pH\*** | **DO** | **CODMn** | **NH3-N** |
| -------------- | -------- | ------ | --------- | --------- |
| 四川攀枝花龙洞 | 7.94     | 9.47   | 1.63      | 0.077     |
| 重庆朱沱       | 8.15     | 9.00   | 1.4       | 0.417     |
| 湖北宜昌南津关 | 8.06     | 8.45   | 2.83      | 0.203     |

![alt text](<imgs/第7章 权重生成与评价模型/image.png>)


In [1]:
import pandas as pd

df = pd.DataFrame(columns=["local", "ph", "DO", "CODMn", "NH3-N"])
df.loc[0] = ["四川攀枝花龙洞", 7.94, 9.47, 1.63, 0.77]
df.loc[1] = ["重庆朱沱", 8.15, 9.0, 1.4, 0.417]
df.loc[2] = ["湖北宜昌南津关", 8.06, 8.45, 2.83, 0.203]
df

,local,ph,DO,CODMn,NH3-N
0,四川攀枝花龙洞,7.94,9.47,1.63,0.770
1,重庆朱沱,8.15,9.00,1.40,0.417
2,湖北宜昌南津关,8.06,8.45,2.83,0.203


首先，我们需要对上面的数据分析：该评价问题一共有三个样本，四个评价指标。不同的评价指标还不太一样，有的越大越好有的越小越好

接下来的操作就是对目标层到准则层构建一个大小为4的方阵，准则层到方案层构建4个大小为3的方阵。我们先来计算一下这个目标层到准则层，至于准则层到方案层的矩阵都是如法炮制的过程。例如，创建了这么一个矩阵：

| **变量**  | **pH\*** | **DO** | **CODMn** | **NH3-N** |
| --------- | -------- | ------ | --------- | --------- |
| **pH\***  | 1        | 1/5    | 1/3       | 1         |
| **DO**    | 5        | 1      | 3         | 5         |
| **CODMn** | 3        | 1/3    | 1         | 3         |
| **NH3-N** | 1        | 1/5    | 1/3       | 1         |


In [2]:
from src.evaluation import AHP
import numpy as np

A = [
    [1, 1 / 5, 1 / 3, 1],
    [5, 1, 3, 5],
    [3, 1 / 3, 1, 3],
    [1, 1 / 5, 1 / 3, 1],
]


# AHP.Criterion 会检测判断矩阵是否符合两条特性
# 并且会自动计算 CR 和 权重
# 如果对计算 CR 和 权重部分 有所修改
# 可以按照 Note 中提示修改
A = AHP.Criterion(level="A", matrix=A)

if A.check_CR():
    print("CR=", A.CR)
    print("对比矩阵A通过一致性检验，各向量权重向量Q为：")
    print(A.W)
else:
    print("对比矩阵A未通过一致性检验，需对对比矩阵A重新构造")

CR= 0.0161086894844068
对比矩阵A通过一致性检验，各向量权重向量Q为：
[0.09546368 0.55958606 0.24948658 0.09546368]


计算出权重向量后，根据相应的测量值表，计算出最终得分

In [3]:
ahp = AHP([A]) # 将所有层次汇聚为一个列表加入。
ahp.run()  # 在run的时候会检测CR，并且融合权重

df["score"] = df.iloc[:, 1:].apply(ahp.calculate_score, axis=1)
df

,local,ph,DO,CODMn,NH3-N,score
0,四川攀枝花龙洞,7.94,9.47,1.63,0.770,6.537432
1,重庆朱沱,8.15,9.00,1.40,0.417,6.203393
2,湖北宜昌南津关,8.06,8.45,2.83,0.203,6.223366


计算出目标层到准则层的1个权重向量和4个准则层到方案层的权重向量以后，可以列出 表5.5 将权重向量进行排布。

 **例7.1中4个权重向量的排布**

| **地点名称**   | **pH\*** | **DO** | **CODMn** | **NH3-N** | **得分** |
| -------------- | -------- | ------ | --------- | --------- | -------- |
|                | 0.0955   | 0.5596 | 0.2495    | 0.0955    |          |
| 四川攀枝花龙洞 | 0.4166   | 0.5396 | 0.2970    | 0.6370    | 0.4767   |
| 重庆朱沱       | 0.3275   | 0.2970 | 0.5396    | 0.1047    | 0.3421   |
| 湖北宜昌南津关 | 0.2599   | 0.1634 | 0.1634    | 0.2583    | 0.1817   |

将准则层到方案层得到的7个成对比较矩阵对应的权重向量排列为一个二维表

- 一行表示对应的方案
- 一列代表评价准则

将这一方案权重矩阵与目标层到准则层的权重向量进行数量积，得到的分数就是最终的评分。

> 最终得到的一个结论是：在评价过程中水中溶解氧含量与钴金属含量占评价体系比重最大，而四川攀枝花龙洞的水质虽然含钴元素比另外两个更高，但由于溶解氧更多，NH3-N的含量更小，水体不显富营养化。就整体而言，四川攀枝花龙洞得分高于重庆朱沱和湖北宜昌南津关。


## 7.2 熵权分析法

熵权法是一种客观赋权方法，基于信息论的理论基础，根据各指标的数据的分散程度，利用信息熵计算并修正得到各指标的熵权，较为客观。相对而言这种数据驱动的方法就回避了上面主观性因素造成的重复修正的影响。


### 7.2.1 什么时信息熵


假设 $x$ 表示事件 $X$ 可能发生的某种情况，$p(x)$ 表示**这种情况发生的概率**我们可以定义 $I(x)=-\ln(p(x))$。因为 $0 \leq p ( x ) \leq 1$，所以 $I(x)\geq 0$。 

如果事件 $X$ 一共有 $x_1, x_2, \cdots, x_n$ 这 $n$ 种情况，那么事件 $X$ 的信息熵为:

$$
H(X)=\sum_{i=1}^n[p(x_i)I(x_i)]=-\sum_{i=1}^n[p(x_i)\ln(p(x_i))]
$$

并且当 $p(x_1)=p(x_1)=\cdots=p(x_n)=1/n$ 时，$H(X)=\ln(n)$，取得最大值。





### 7.2.1 熵权分析法的原理

衡量方案好坏的指标有多种性质，有些指标越大越好，称为效益性指标；有些指标越小越好，称为成本型指标；有些指标越接近特定值越好，称为目标性指标；有些指标越接近特定区间越好，称为范围性指标。

若模型未经数据预处理直接计算，可能出现问题。因此，第一步是指标正向化。它涉及将原本的负向指标转为正向指标，例如将死亡率转为生存率、故障率转为可靠度。通过正向化，目标或结果的达成情况更直观，提高指标的可解释性和可操作性。




#### 正向化

*所谓正向化就是将指标值转换效益指标*

- 对于效益型指标：不需要正向化。只需要通过 min-max规约 或 Z-score规约 进行规约即可。
- 对于成本型指标：它的正向化方式比较简单，可以取相反数；如果指标全部为正数，也可以取其倒数。
- 对于区间型指标：它的规约方法遵循下面的式子。
    $$
    x_{new}=
    \begin{cases}
    1-\frac{a-x}{{M}} &, x < a\\
    1 &, a \le x \le b\\
    1-\frac{x-b}{M} &, x > b
    \end{cases}
    $$
- 对于中值型指标：它的正向化操作为
    $$x_{new}=1-\frac{\left|x-x\right._{best}|}{max\left(\left|x-x\right._{best}\right|)}$$


在进行指标正向化以后，所有的指标全部被变换**效益指标**。而为了进一步消除量纲这些的影响，需要进一步进行min-max规约化或z-score规约化消除量纲差异。


#### 熵权法的主要计算步骤如下

1. 构建 $m$ 个事物 $n$ 个指标的矩阵 $X_{m\times n}$
2. 将判断矩阵进行归一化处理，得到新的归一化判断矩阵$X_{m\times n}$。
    $$b_{ij}=\frac{x_{ij}-\min ( x_{j} ) }{\max ( x_{j} ) - \min ( x_{j} )  }$$
3. 计算每个评价指标的权重矩阵 $P_{m\times n}$
   $$p_{ij}=\frac{b_{ij}}{\sum_{i=1}^{m}b_{ij}}$$
4. 根据信息论中对熵的定义，计算熵值向量 $E_{n}$
   $$e_j=-\frac{1}{\ln m}\sum_{i=1}^mp_{ij}\ln p_{ij}$$
5. 计算每个指标的权重系数向量 $W_n$
    $$w_j=\frac{1-e_j}{\sum_{i=1}^n(1-e_j)}$$
> 注意：熵权法是一个数据驱动过程，一定要保证有一定数据量并且做了正向化。

### 7.2.2 熵权分析法的案例

继续上一个水样抽样的模型

| **地点名称** | **pH\*** | **DO** | **CODMn** | **NH3-N** | **鱼类密度** | **垃圾密度** |
| :----------------- | -------------- | ------------ | --------------- | --------------- | ------------------ | ------------------ |
| 四川攀枝花龙洞     | 7.94           | 9.47         | 1.63            | 0.08            | 6.78               | 4.94               |
| 重庆朱沱           | 8.15           | 9.00         | 1.40            | 0.42            | 5.27               | 4.47               |
| 湖北宜昌南津关     | 8.06           | 8.45         | 2.83            | 0.20            | 2.50               | 7.03               |
| 湖南岳阳城陵矶     | 8.05           | 9.16         | 3.33            | 0.29            | 5.65               | 5.26               |
| 江西九江河西水厂   | 7.60           | 7.93         | 2.07            | 0.13            | 5.26               | 6.39               |
| 安徽安庆皖河口     | 7.39           | 7.12         | 2.23            | 0.20            | 6.21               | 7.50               |
| 江苏南京林山       | 7.74           | 7.29         | 1.77            | 0.06            | 6.44               | 3.93               |
| 四川乐山岷江大桥   | 7.38           | 6.51         | 3.63            | 0.41            | 3.17               | 5.75               |
| 四川宜宾凉姜沟     | 8.32           | 8.47         | 1.60            | 0.14            | 3.32               | 6.29               |
| 四川泸州沱江二桥   | 7.69           | 8.50         | 2.73            | 0.28            | 7.25               | 8.21               |
| 湖北丹江口胡家岭   | 8.15           | 9.88         | 2.00            | 0.08            | 7.22               | 3.82               |
| 湖南长沙新港       | 6.88           | 7.59         | 1.77            | 0.92            | 2.94               | 8.03               |
| 湖南岳阳岳阳楼     | 8.00           | 8.15         | 4.87            | 0.33            | 4.68               | 5.01               |
| 湖北武汉宗关       | 7.94           | 7.48         | 3.30            | 0.13            | 5.81               | 4.87               |
| 江西南昌滁槎       | 8.01           | 7.76         | 2.67            | 6.36            | 4.52               | 3.22               |
| 江西九江蛤蟆石     | 7.91           | 7.93         | 5.47            | 0.21            | 7.48               | 2.40               |
| 江苏扬州三江营     | 8.04           | 8.34         | 3.87            | 0.20            | 3.73               | 4.05               |


首先，读取数据并对指标进行正向化。

pH这一列虽然是区间型指标，但是可以用pH=7作为最优值将其看作中间值型指标处理，与7的偏差越大则得分越小。DO和鱼类密度是极大型指标，剩下三个是极小型指标，所以用倒数的方法正向化。

In [4]:
import pandas as pd
import numpy as np

data=pd.read_csv("./notebooks/data/test.csv")
data.head()

,地点名称,pH,DO,CODMn,NH3-N,鱼类密度,垃圾密度
0,四川攀枝花龙洞,7.94,9.47,1.63,0.08,6.78,4.94
1,重庆朱沱,8.15,9.00,1.40,0.42,5.27,4.47
2,湖北宜昌南津关,8.06,8.45,2.83,0.20,2.50,7.03
3,湖南岳阳城陵矶,8.05,9.16,3.33,0.29,5.65,5.26
4,江西九江河西水厂,7.60,7.93,2.07,0.13,5.26,6.39


In [5]:
def target_norm(x: np.ndarray, target):
    """目标指标正向化"""
    return 1 - np.abs(x - target) / np.max(np.abs(x - target))


def cost_norm(x: np.ndarray):
    """
    成本指标正向化
    """
    return 1 / x


In [6]:
data["pH"] = target_norm(data["pH"], 7)

data["CODMn"] = cost_norm(data["CODMn"])
data["NH3-N"] = cost_norm(data["NH3-N"])
data["垃圾密度"] = cost_norm(data["垃圾密度"])

X=data.drop(columns=["地点名称"])
X

,pH,DO,CODMn,NH3-N,鱼类密度,垃圾密度
0,0.287879,9.47,0.613497,12.500000,6.78,0.202429
1,0.128788,9.00,0.714286,2.380952,5.27,0.223714
2,0.196970,8.45,0.353357,5.000000,2.50,0.142248
3,0.204545,9.16,0.300300,3.448276,5.65,0.190114
4,0.545455,7.93,0.483092,7.692308,5.26,0.156495
5,0.704545,7.12,0.448430,5.000000,6.21,0.133333
6,0.439394,7.29,0.564972,16.666667,6.44,0.254453
7,0.712121,6.51,0.275482,2.439024,3.17,0.173913
8,0.000000,8.47,0.625000,7.142857,3.32,0.158983
9,0.477273,8.50,0.366300,3.571429,7.25,0.121803


In [7]:
def normalize(X: np.ndarray):
    return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))


def entropy_weight_analysis(X: np.ndarray):
    col_max = np.max(X, axis=0)
    col_min = np.min(X, axis=0)
    B = (X - col_min) / (col_max - col_min)
    P = (1+B) / np.sum((1+B), axis=0)
    m = P.shape[0]
    E = (-1 / np.log(m)) * np.sum(P * np.log(P), axis=0)
    W = (1 - E) / np.sum(1 - E, axis=0)
    return W

In [8]:
entropy_weight_analysis(X)

pH       0.155464
DO       0.132337
CODMn    0.186279
NH3-N    0.164323
鱼类密度     0.199353
垃圾密度     0.162245
dtype: float64

### 7.3.1 TOPSIS分析法

TOPSIS法（Technique for Order Preference by Similarity to Ideal Solution）可翻译为逼近理想解排序法，国内常简称为优劣解距离法。

在TOPSIS分析法中，我们通过计算每个方案离理想解和负理想解的距离来判断优劣。理想解是最佳方案，各项指标最优；负理想解是最差方案，各项指标最差。这种方法就像“近朱者赤近墨者黑”，离最好的方案越近，表现就越优秀，离最差的方案越远，表现就越差。


#### 计算距离的多种方式

欧几里得距离、曼哈顿距离、余弦距离。

$$Euclidean\left(x,y\right)=\sqrt{\sum_{i=1}^n{(x_i-y_i)^2}}$$

$$Manhaton\left(x,y\right)=\sum_{i=1}^n|x_i-y_i|$$

$$cos\left(x,y\right)=\frac{x^Ty}{|x||y|}$$


可以用来计算距离得分，记为 $distance(x,y)$